In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pandas as pd
import numpy as np
import os
import json

In [11]:
judge_llama_folder = "/content/drive/MyDrive/Portfolio Projects/Mestrado/Outputs/results/judge_llama/contextualized/"
files = os.listdir(judge_llama_folder)
#files = files[:5]

df_judge_llama = pd.DataFrame()

for file in files:
    filename = file.split(".")[0]
    #print(file)

    if file.endswith(".json"):
        dff = pd.read_json(judge_llama_folder + file)
        dff = dff.loc["score"].reset_index().T
        dff.columns = dff.iloc[0]
        dff = dff.drop(dff.index[0]).reset_index().drop("index",axis=1)
        dff["doc"] = filename
        #print(filename, dff)
        df_judge_llama = pd.concat([df_judge_llama, dff])

df_judge_llama.reset_index(inplace=True)
df_judge_llama = df_judge_llama.drop("index",axis=1)
df_judge_llama = df_judge_llama[["doc", "chatgpt", "qwen"]]
df_judge_llama["doc"] = df_judge_llama["doc"].apply(lambda x: x.split("_")[0])

In [12]:
judge_qwen_folder = "/content/drive/MyDrive/Portfolio Projects/Mestrado/Outputs/results/judge_qwen/contextualized/"
files = os.listdir(judge_qwen_folder)
#files = files[:5]

df_judge_qwen = pd.DataFrame()

for file in files:
    filename = file.split(".")[0]
    #print(file)

    if file.endswith(".json"):
        dff = pd.read_json(judge_qwen_folder + file)
        dff = dff.loc["score"].reset_index().T
        dff.columns = dff.iloc[0]
        dff = dff.drop(dff.index[0]).reset_index().drop("index",axis=1)
        dff["doc"] = filename
        #print(filename, dff)
        df_judge_qwen = pd.concat([df_judge_qwen, dff])

df_judge_qwen.reset_index(inplace=True)
df_judge_qwen = df_judge_qwen.drop("index",axis=1)
df_judge_qwen = df_judge_qwen[["doc", "chatgpt", "llama"]]
df_judge_qwen["doc"] = df_judge_qwen["doc"].apply(lambda x: x.split("_")[0])

In [13]:
judge_chatgpt_folder = "/content/drive/MyDrive/Portfolio Projects/Mestrado/Outputs/results/judge_chatgpt/contextualized/"
files = os.listdir(judge_chatgpt_folder)
#files = files[:5]

df_judge_chatgpt = pd.DataFrame()

for file in files:
    filename = file.split(".")[0]
    #print(file)

    if file.endswith(".json"):
        dff = pd.read_json(judge_chatgpt_folder + file)
        dff = dff.loc["score"].reset_index().T
        dff.columns = dff.iloc[0]
        dff = dff.drop(dff.index[0]).reset_index().drop("index",axis=1)
        dff["doc"] = filename
        #print(filename, dff)
        df_judge_chatgpt = pd.concat([df_judge_chatgpt, dff])

df_judge_chatgpt.reset_index(inplace=True)
df_judge_chatgpt = df_judge_chatgpt.drop("index",axis=1)
df_judge_chatgpt = df_judge_chatgpt[["doc", "qwen", "llama"]]
df_judge_chatgpt["doc"] = df_judge_chatgpt["doc"].apply(lambda x: x.split("_")[0])

In [14]:
df_judge_chatgpt.rename(columns={"llama":"llama_judge_chatgpt", "qwen":"qwen_judge_chatgpt"},inplace=True)
df_judge_llama.rename(columns={"chatgpt":"chatgpt_judge_llama", "qwen":"qwen_judge_llama"},inplace=True)
df_judge_qwen.rename(columns={"llama":"llama_judge_qwen", "chatgpt":"chatgpt_judge_qwen"},inplace=True)

In [15]:
df_judges = df_judge_chatgpt.merge(df_judge_llama, how = "left", on = "doc")
df_judges =        df_judges.merge(df_judge_qwen, how = "left", on = "doc")

In [20]:
for i in df_judges.iloc[:,1:].columns:
    print(i, df_judges[i].unique())

qwen_judge_chatgpt [0 4 1 7 6 2 3 None 8 5 9]
llama_judge_chatgpt [0 6 3 4 7 9 8 2 5 1 None]
chatgpt_judge_llama [0 8 9 7 6]
qwen_judge_llama [0 8 9 7 nan 6 2]
chatgpt_judge_qwen [0 9 8 7 5 6 3 4]
llama_judge_qwen [0 9 8 7 5 6 3 2 4]


In [21]:
def compute_mean_or_value(df, col_a, col_b):
    return df.apply(
        lambda x: (int(x[col_a]) + int(x[col_b])) / 2 if pd.notnull(x[col_a]) and pd.notnull(x[col_b])
        else int(x[col_a]) if pd.notnull(x[col_a])
        else int(x[col_b]) if pd.notnull(x[col_b])
        else np.nan,
        axis=1
    )
df_judges["llama_score"] = compute_mean_or_value(df_judges, "llama_judge_chatgpt", "llama_judge_qwen")
df_judges["qwen_score"] = compute_mean_or_value(df_judges, "qwen_judge_chatgpt", "qwen_judge_llama")
df_judges["chatgpt_score"] = compute_mean_or_value(df_judges, "chatgpt_judge_llama", "chatgpt_judge_qwen")

In [22]:
for col in df_judges.iloc[:,1:].columns:
    print(col, round(df_judges[col].mean(),2))

qwen_judge_chatgpt 3.72
llama_judge_chatgpt 3.25
chatgpt_judge_llama 8.03
qwen_judge_llama 8.16
chatgpt_judge_qwen 7.71
llama_judge_qwen 7.56
llama_score 5.41
qwen_score 5.69
chatgpt_score 7.87


In [23]:
dff = df_judges.copy()
dff["judge_chatgpt"] = (dff["qwen_judge_chatgpt"] + dff["llama_judge_chatgpt"]) / 2
dff["judge_llama"]   = (dff["qwen_judge_llama"] + dff["chatgpt_judge_llama"]  ) / 2
dff["judge_qwen"]    = (dff["llama_judge_qwen"] + dff["llama_judge_qwen"]     ) / 2

lis = ["judge_chatgpt","judge_llama","judge_qwen"]
for i in lis:
    print(i, dff[i].mean())

judge_chatgpt 3.4716216216216216
judge_llama 8.095166163141993
judge_qwen 7.562666666666667


In [ ]:
dff[lis].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   judge_chatgpt  372 non-null    object
 1   judge_llama    331 non-null    object
 2   judge_qwen     375 non-null    object
dtypes: object(3)
memory usage: 8.9+ KB


In [24]:
lis = ["judge_chatgpt", "judge_llama", "judge_qwen"]
for col in lis:
    null_count = sum(dff[col].isna())
    total_count = len(dff)
    null_perc = null_count / total_count
    print(col, round(null_perc,2) * 100)

judge_chatgpt 1.0
judge_llama 12.0
judge_qwen 0.0


In [25]:
lis = ["chatgpt_score", "llama_score", "qwen_score"]
for col in lis:
    print(col, df_judges[col].mean())

chatgpt_score 7.868
llama_score 5.410666666666667
qwen_score 5.6866666666666665


## Explanations

In [48]:
judge = 'chatgpt'
model = 'llama'
file_path = f"/content/drive/MyDrive/Portfolio Projects/Mestrado/Outputs/results/judge_{judge}/full_model_assessments/{model}_assessment.json"
with open(file_path, 'r') as f:
  assessment = json.load(f)

for i in assessment:
  print(i)#, assessment[i])
  if i != 'summary':
    for j in assessment[i]:
      print(j)
  else:
    print(assessment[i])

strengths
Capacidade de formular respostas com conteúdo relevante em alguns contextos
Seguimento de instruções básicas de formatação como uso de hífens em listas
Concisão em respostas quando a tarefa é claramente compreendida
weaknesses
Inconsistência na aderência ao formato detalhado e especificidade do conteúdo solicitado
Falta de detalhamento e precisão na escolha de tópicos relevantes e específicos do texto fornecido
Dificuldade em compreender e seguir instruções complexas de tarefas
Tendência a fornecer respostas genéricas ou vagas que não capturam a essência dos temas discutidos
recommendations
Melhorar a compreensão das instruções detalhadas, garantindo a aderência exata ao formato e especificidade solicitada
Aumentar a precisão na seleção de tópicos, assegurando que os tópicos escolhidos sejam diretamente relevantes para o conteúdo discutido
Treinar o modelo para evitar generalizações e focar em detalhes chave mencionados nos prompts
Desenvolver uma melhor capacidade de análise

In [53]:
judge = 'llama'
model = 'chatgpt'
file_path = f"/content/drive/MyDrive/Portfolio Projects/Mestrado/Outputs/results/judge_{judge}/model_assessments/{model}_assessment.json"
with open(file_path, 'r') as f:
  assessment = json.load(f)

for i in assessment:
  print(i)#, assessment[i])
  if i != 'summary':
    for j in assessment[i]:
      print(j)
  else:
    print(assessment[i])

strengths
O modelo apresentou uma boa compreensão dos tópicos relevantes para cada texto original, atendendo aos limites estabelecidos e sendo concisos.
As avaliações da percepção do público foram feitas com base nas respostas dos participantes, mostrando-se satisfeitos e positivos em relação ao apresentado.
O modelo conseguiu capturar os aspectos-chave das discussões sobre as instituições financeiras, como custo de crédito, estratégias de provisões financeiras e crescimento de serviços digitais.
weaknesses
Em alguns casos, o modelo não considerou todos os pontos importantes do texto original, como a menção à captura de valor no Banco Pan.
A avaliação da percepção do público pode ter sido influenciada pelas respostas dos participantes e não necessariamente refletiu a opinião geral do público.
Houve omissões em relação aos aspectos negativos das instituições financeiras, como questões financeiras desfavoráveis ou desempenho operacional ruim.
recommendations
Melhorar a compreensão e incl

In [54]:
judge = 'llama'
model = 'qwen'
file_path = f"/content/drive/MyDrive/Portfolio Projects/Mestrado/Outputs/results/judge_{judge}/model_assessments/{model}_assessment.json"
with open(file_path, 'r') as f:
  assessment = json.load(f)

for i in assessment:
  print(i)#, assessment[i])
  if i != 'summary':
    for j in assessment[i]:
      print(j)
  else:
    print(assessment[i])

strengths
O modelo foi capaz de extrair as informações necessárias para as duas tarefas solicitadas.
A lista de tópicos (Tarefa 1) foi limitada a 10 itens, como solicitado, e cada item tinha no máximo 5 palavras.
A resposta foi clara e direta, fornecendo uma percepção positiva da apresentação para a Tarefa 2.
weaknesses
Algumas respostas não atenderam às especificações de formatação ou detalhamento solicitados.
Não houve referências explícitas ao resultado do seminário para inadimplência e desempenho no mercado em 2023 nas perguntas ou respostas.
recommendations
Melhorar a precisão e detalhamento das informações fornecidas, especialmente em casos onde as especificações de formatação foram ignoradas.
Incluir tópicos adicionais relevantes para abranger todas as partes do texto
Aprimorar a habilidade de capturar as informações necessárias para as duas tarefas solicitadas
summary
O modelo apresentou uma boa performance, mas com alguns pontos fracos em termos de precisão e detalhamento. Rec

In [55]:
judge = 'qwen'
model = 'chatgpt'
file_path = f"/content/drive/MyDrive/Portfolio Projects/Mestrado/Outputs/results/judge_{judge}/model_assessments/{model}_assessment.json"
with open(file_path, 'r') as f:
  assessment = json.load(f)

for i in assessment:
  print(i)#, assessment[i])
  if i != 'summary':
    for j in assessment[i]:
      print(j)
  else:
    print(assessment[i])

strengths
Identificação correta dos tópicos principais discutidos
Resposta adequada à primeira tarefa de identificar os tópicos mais importantes
Avaliação correta das perguntas do público com a palavra 'positivo'
weaknesses
Falta de detalhes nos tópicos apresentados
Não seguiu exatamente o formato solicitado (como não iniciar com 'Tarefa x:' e sem usar '-' nos tópicos)
Pouco contexto ou explicação sobre os tópicos discutidos
recommendations
Incluir mais detalhes e explicações nos tópicos
Manter o formato solicitado ao responder as tarefas
Adicionar um breve resumo final que aborde todos os tópicos mencionados
summary
O modelo apresentou uma resposta adequada à solicitação, identificando corretamente os tópicos principais discutidos e avaliando positivamente as perguntas do público. No entanto, existem áreas para melhoria na inclusão de mais detalhes, manutenção de um formato específico e fornecimento de contexto ou explicação em torno dos tópicos apresentados.


In [56]:
judge = 'qwen'
model = 'llama'
file_path = f"/content/drive/MyDrive/Portfolio Projects/Mestrado/Outputs/results/judge_{judge}/model_assessments/{model}_assessment.json"
with open(file_path, 'r') as f:
  assessment = json.load(f)

for i in assessment:
  print(i)#, assessment[i])
  if i != 'summary':
    for j in assessment[i]:
      print(j)
  else:
    print(assessment[i])

strengths
Respondeu todas as tarefas com precisão e concisão
Identificou todos os pontos principais discutidos no texto ou áudio
Manter uma pontuação limitada a cinco palavras por tópico conforme solicitado
Avaliação da percepção do público foi correta
weaknesses
Falta de detalhes adicionais que poderiam dar mais contexto e riqueza aos tópicos
Algumas respostas excederam o limite máximo de 10 tópicos
A ausência de justificativas ou explicações mais detalhadas para algumas ações ou estratégias discutidas
Não houve consideração completa para todas as questões ou aspectos da transcrição em cada resposta
recommendations
Poderia fornecer mais detalhes para melhor esclarecer e explicar os tópicos principais
Deve respeitar exatamente o limite de 10 tópicos por solicitação, evitando a inclusão desnecessária de itens adicionais
Melhor entender completamente a transcrição ou texto para responder todas as questões e aspectos necessários
Pode considerar justificar ou explicar os termos ou ideias t